In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import yfinance as yf
import time
from tqdm import tqdm
import collections
from db_connector import connect_db
from db_connector import engine_db

if not hasattr(collections, 'Callable'):
    collections.Callable = collections.abc.Callable

engine = engine_db()
con = connect_db()

mycursor = con.cursor()

ticker_list = pd.read_sql("""
select * from global_ticker
where date = (select max(date) from global_ticker)
and country = 'United States'
and (`Market Cap` like '%%B' and length(`Market Cap`) > 6
OR (`Market Cap` like '%%T'));                          
""", con=engine)

truncate_table = """
truncate table global_price;
"""

mycursor.execute(truncate_table)
con.commit()

query = """
    insert into global_price (Date, High, Low, Open, Close, Volume, `Adj Close`, ticker)
    values (%s, %s,%s,%s,%s,%s,%s,%s) as new
    on duplicate key update
    High = new.High, Low = new.Low, Open = new.Open, Close = new.Close,
    Volume = new.Volume, `Adj Close` = new.`Adj Close`;
"""

error_list = []

for i in tqdm(range(0, len(ticker_list))):

    ticker = ticker_list['Symbol'][i]

    try:

        price = yf.download(ticker, progress=False, period='7d')

        price = price.reset_index()
        price['ticker'] = ticker

        args = price.values.tolist()
        mycursor.executemany(query, args)
        con.commit()

    except:

        print(ticker)
        error_list.append(ticker)

    time.sleep(2)

engine.dispose()
con.close()

 18%|█▊        | 17/96 [00:40<03:06,  2.36s/it]
1 Failed download:
['BRKA']: Exception('%ticker%: No data found, symbol may be delisted')
 19%|█▉        | 18/96 [00:46<04:21,  3.35s/it]
1 Failed download:
['BRKB']: Exception('%ticker%: No data found, symbol may be delisted')
100%|██████████| 96/96 [03:54<00:00,  2.44s/it]
